Install system dependencies

In [1]:
!apt-get install -y ffmpeg xvfb -q

Reading package lists...
Building dependency tree...
Reading state information...
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 784 kB of archives.
After this operation, 2,271 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.10 [784 kB]
Fetched 784 kB in 0s (2,822 kB/s)
Selecting previously unselected package xvfb.
(Reading database ... 155632 files and directories currently installed.)
Preparing to unpack .../xvfb_2%3a1.19.6-1ubuntu4.10_amd64.deb ...
Unpacking xvfb (2:1.19.6-1ubuntu4.10) ...
Setting up xvfb (2:1.19.6-1ubuntu4.10) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [2]:
!pip install -U pyglet colabgymrender pyvirtualdisplay pyglet gym gym[classic_control] --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.0 MB/s 
     |████████████████████████████████| 696 kB 43.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


Set up display on headless remote machine

In [8]:
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [9]:
def get_latest_mp4(folder):
  mp4_names = [name for name in os.listdir(folder) if name.endswith(".mp4") ]
  mp4_names.sort(key=lambda x: x.split('.')[-2], reverse=True)
  return os.path.join(folder, mp4_names[0])

def show_video(mp4_path):
  from IPython.display import HTML
  from base64 import b64encode
  mp4 = open(mp4_path,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width=400 autoplay controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url)

In [10]:
import gym.wrappers
import random
from colabgymrender.recorder import Recorder

We can watch a video of the environment taking random actions  [documentation for cartpole](https://www.gymlibrary.ml/environments/classic_control/cart_pole/)

In [17]:
env = gym.make("CartPole-v1")
env = gym.wrappers.Monitor(env, './videos/random', force=True)

obs = env.reset()
done = False
while not done:
    # take random actions
    obs, rew, done, info = env.step(random.randint(0, 1)) # alternatively, we could use env.step(env.action_space.sample())
    
show_video(get_latest_mp4("./videos/random"))

We can also measure how good of a job it is doing

In [12]:
env = gym.make("CartPole-v1")

total_reward = 0
num_episodes = 1000
for i in range(num_episodes):
  episodic_reward = 0
  obs = env.reset()
  done = False
  while not done:
      # take random actions
      obs, rew, done, info = env.step(random.randint(0, 1)) # alternatively, we could use env.step(env.action_space.sample())
      total_reward += rew

total_reward / num_episodes

22.342

Now we can try to make a better policy than random


In [1]:
env = gym.make("CartPole-v1")
env = gym.wrappers.Monitor(env, './videos/manual', force=True)

MOVE_LEFT = 0
MOVE_RIGHT = 1

def manual_policy(obs):
  pos, vel, ang, angvel = obs
  return MOVE_LEFT if ang <= 0 else MOVE_RIGHT

total_reward = 0
num_episodes = 100
for i in range(num_episodes):
  episodic_reward = 0
  obs = env.reset()
  done = False
  while not done:
      # take random actions
      obs, rew, done, info = env.step(manual_policy(obs))
      total_reward += rew
    
print(total_reward/num_episodes)
show_video(get_latest_mp4("videos/manual"))

NameError: ignored

In [ ]:
import numpy as np

env = gym.make("CartPole-v1")
env = gym.wrappers.RecordVideo(env, './videos/matrix_manual')

MOVE_LEFT = 0
MOVE_RIGHT = 1

matrix_policy = np.array([
   [0, 0, 10, 0],
])

def manual_policy(obs):
  logits = matrix_policy @ obs
  prob_left = 1 / (1+np.exp(logits))
  return MOVE_LEFT if random.random() < prob_left else MOVE_RIGHT


total_reward = 0
num_episodes = 100
for i in range(num_episodes):
  episodic_reward = 0
  obs = env.reset()
  done = False
  while not done:
      # take random actions
      obs, rew, done, info = env.step(manual_policy(obs))
      total_reward += rew
    
print(total_reward/num_episodes)
show_video(get_latest_mp4("videos/manual"))

42.61


Policy gradient theorem
-